In [1]:
import torch
from torch.optim import Adam
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from torchsummary import summary
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

from LoadDataset.utils import (
    MaskDetectionDataSet,
    dataset_split,
    dataloader
)


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
device

device(type='cuda')

In [3]:
torch.manual_seed(100)

In [4]:
data_dir = "../data"

In [5]:
dataset = MaskDetectionDataSet(data_dir)

In [6]:
dataset.classes

['with_mask', 'without_mask']

In [7]:
train_ds, valid_ds = dataset_split(dataset, val_ratio = 0.2)

In [8]:
train_dl = dataloader(train_ds, batch_size=16, shuffle=True)
valid_dl = dataloader(valid_ds, batch_size=16, shuffle=False)

In [9]:
for img, labels in train_dl:
    print(img.shape)
    print(labels.shape)
    break

torch.Size([16, 3, 32, 32])
torch.Size([16])


Transfer Learning using pre-trained ResNet34 model

In [10]:
class MaskModel(nn.Module):
    def __init__(self, num_classes, pretrained = True):
        super().__init__()
        # use pretrained model
        self.network = models.resnet34(pretrained=pretrained)
        # Replace Last layer
        self.network.fc = nn.Linear(self.network.fc.in_features, num_classes)

    def forward(self, x):
        return self.network(x)

In [11]:
model = MaskModel(len(dataset.classes), pretrained=True).to(device)

In [12]:
summary(model.to(device), input_size=(3, 32, 32), batch_size = 16)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [16, 64, 16, 16]           9,408
       BatchNorm2d-2           [16, 64, 16, 16]             128
              ReLU-3           [16, 64, 16, 16]               0
         MaxPool2d-4             [16, 64, 8, 8]               0
            Conv2d-5             [16, 64, 8, 8]          36,864
       BatchNorm2d-6             [16, 64, 8, 8]             128
              ReLU-7             [16, 64, 8, 8]               0
            Conv2d-8             [16, 64, 8, 8]          36,864
       BatchNorm2d-9             [16, 64, 8, 8]             128
             ReLU-10             [16, 64, 8, 8]               0
       BasicBlock-11             [16, 64, 8, 8]               0
           Conv2d-12             [16, 64, 8, 8]          36,864
      BatchNorm2d-13             [16, 64, 8, 8]             128
             ReLU-14             [16, 6

Optimizer

In [13]:
optimizer = Adam(model.parameters())

Loss function

cross entroy loss

In [14]:
loss_fn = nn.CrossEntropyLoss()

Accuracy

In [15]:
def accuracy(y_true, y_pred):
    return torch.tensor(torch.sum(y_true==y_pred).item()/len(y_pred))

In [16]:
model.train()
training_loss = []
validation_loss = []
training_accuracy = []
validation_accuracy = []
epoch = 2
for i in range(epoch):
    train_loss = []
    val_loss = []
    train_accuracy = []
    val_accuracy = []
    train_loop = tqdm(train_dl, leave=True)
    for x, labels in train_loop:
        train_loop.set_description(f"Epoch {i+1}")
        optimizer.zero_grad()
        y = model(x)
        loss = loss_fn(y.float(), labels)
        loss.backward()
        optimizer.step()
        _, pred =torch.max(y, dim = 1)
        accuracy_val = accuracy(labels, pred)
        train_loss.append(loss.item())
        train_accuracy.append(accuracy_val.item())

        train_loop.set_postfix(
            train_loss=sum(train_loss) / len(train_loss),
            train_accuracy=sum(train_accuracy) / len(train_accuracy),
        )

    val_loop = tqdm(valid_dl, leave=True)
    with torch.no_grad():
        for x, labels in val_loop:
            y = model(x)
            loss = loss_fn(y.float(), labels)

            _, pred =torch.max(y, dim = 1)
            accuracy_val = accuracy(labels, pred)
            val_loss.append(loss.item())
            val_accuracy.append(accuracy_val.item())

            val_loop.set_postfix(
                train_loss=sum(train_loss) / len(train_loss),
                train_accuracy=sum(train_accuracy) / len(train_accuracy),
                val_loss=sum(val_loss) / len(val_loss),
                val_accuracy=sum(val_accuracy) / len(val_accuracy),
            )

    training_loss.append(sum(train_loss) / len(train_loss))
    training_accuracy.append(sum(train_accuracy) / len(train_accuracy))
    validation_loss.append(sum(val_loss) / len(val_loss))
    validation_accuracy.append(sum(val_accuracy) / len(val_accuracy))

100%|██████████| 95/95 [00:18<00:00,  5.07it/s, train_accuracy=0.929, train_loss=0.197, val_accuracy=0.861, val_loss=0.315]


In [17]:
torch.save(model, "../model.pt")